# Fraud Detection in Electricity and Gas Consumption Challenge

This notebook covers the creation of the prediction file to hand in to Zindy!

In [6]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
import sys
sys.path.append("../")


from imblearn.over_sampling import SMOTE

# Preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder
from src.preprocessing.agg_invoice import agg_invoice_num_mode_monthly_weighting, agg_invoice_smart
from src.preprocessing.cleaning import convert_dtype_cat_date



from sklearn.svm import SVC
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder

from matplotlib.ticker import PercentFormatter
plt.rcParams.update({ "figure.figsize" : (8, 5),"axes.facecolor" : "white", "axes.edgecolor":  "black"})
plt.rcParams["figure.facecolor"]= "w"
pd.plotting.register_matplotlib_converters()
pd.set_option('display.float_format', lambda x: '%.3f' % x)


# Define a plotting style to be used for all plots in this notebook
plt.style.use('tableau-colorblind10')
sns.set_palette("colorblind")

#from sqalchemy import create_engine
#from dotenv import load_dotenv

In [29]:
df_client_train = pd.read_csv('../data/train/client_train.csv')
df_invoice_train = pd.read_csv('../data/train/invoice_train.csv')

In [32]:
df_invoice_train.shape, df_client_train.shape

((4476749, 16), (135493, 6))

In [146]:
with open('../data/data_ready_for_models_smart_agg.pkl', 'rb') as f:
    data = pickle.load(f)
df_combined_encoded_train = data["df_combined_encoded"]
df_combined_train = data["df_combined"]
X_train= data["X_train"]


In [147]:
X_train

,creation_date,invoice_date,months_number,counter_coefficient,consommation_level_1_elec,consommation_level_2_elec,consommation_level_3_elec,consommation_level_4_elec,consommation_level_1_gaz,consommation_level_2_gaz,...,counter_code_483.0,counter_code_506.0,counter_code_532.0,counter_code_565.0,counter_code_600.0,reading_remarque_7.0,reading_remarque_8.0,reading_remarque_9.0,reading_remarque_203.0,reading_remarque_413.0
client_id,,,,,,,,,,,,,,,,,,,,,
train_Client_46032,14510016.000,14606784.000,41,1.000,199.975,86.575,130.925,258.650,123.650,0.000,...,False,False,False,False,False,False,False,True,False,False
train_Client_8884,12896064.000,12939264.000,29,1.000,239.071,99.821,0.000,0.000,0.000,0.000,...,False,False,False,False,False,False,False,False,False,False
train_Client_80860,12251520.000,12052800.000,134,1.000,98.692,0.000,0.000,0.000,0.000,0.000,...,False,False,False,False,False,False,False,False,False,False
train_Client_21716,4678560.000,11249280.000,169,1.000,127.942,0.000,0.000,0.000,19.957,0.000,...,False,False,False,False,False,False,False,True,False,False
train_Client_95171,6067008.000,11237184.000,167,1.000,91.104,0.000,0.000,0.000,20.784,0.000,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
train_Client_107750,8799840.000,11054016.000,174,1.000,249.786,53.714,6.018,0.000,0.000,0.000,...,False,False,False,False,False,False,False,False,False,False
train_Client_59063,8974368.000,11315808.000,162,1.000,115.268,0.000,0.000,0.000,33.074,0.000,...,False,False,False,False,False,False,False,False,False,False
train_Client_68496,12158208.000,12163392.000,137,1.000,107.015,0.000,0.000,0.000,0.000,0.000,...,False,False,False,False,False,False,False,False,False,False


In [45]:
df_combined_encoded_train.shape

(135493, 109)

In [41]:
df_combined_train.shape

(135493, 19)

In [46]:
df_client_test = pd.read_csv('../data/test/client_test.csv')
df_invoice_test = pd.read_csv('../data/test/invoice_test.csv')

df_client_test.shape, df_invoice_test.shape

((58069, 5), (1939730, 16))

In [47]:
# Convert date time format

df_invoice_test['invoice_date'] = pd.to_datetime(df_invoice_test['invoice_date'])
df_client_test['creation_date'] = pd.to_datetime(df_client_test['creation_date'])


In [48]:
# Feature engineering (aggregate)

df_invoice= agg_invoice_smart(df_invoice_test)

In [49]:
# merge dataframes

df_combined = pd.merge(df_client_test, df_invoice, on='client_id', how='inner')


In [50]:
df_combined.shape

(58069, 20)

In [51]:
def convert_dtype_cat_date(df: pd.DataFrame) -> pd.DataFrame:
    df_new = df.copy()
    convert_to_category = [
        "disrict",
        "region",
        "client_catg",
        "tarif_type",
        "counter_statue",
        "counter_code",
        "reading_remarque",
    ]
    convert_to_datetime = ["creation_date", "invoice_date"]
    for category in convert_to_category:
        df_new[category] = df_new[category].astype("category")

    df_new = df_new.rename(columns={"disrict": "district"})

    return df_new

In [52]:
# call cleaning function (convert categories, rename columns, etc.)

df_combined = convert_dtype_cat_date(df_combined) 

In [79]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58069 entries, 0 to 58068
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   district                   58069 non-null  category      
 1   client_id                  58069 non-null  object        
 2   client_catg                58069 non-null  category      
 3   region                     58069 non-null  category      
 4   creation_date              58069 non-null  datetime64[ns]
 5   invoice_date               58069 non-null  datetime64[ns]
 6   months_number              58069 non-null  int64         
 7   counter_coefficient        58069 non-null  float64       
 8   tarif_type                 58069 non-null  category      
 9   counter_statue             58069 non-null  category      
 10  counter_code               58069 non-null  category      
 11  reading_remarque           58069 non-null  category      
 12  cons

In [54]:
# drop consomnation level gaz 3 and 4, since they are always 0
df_combined = df_combined.drop(['consommation_level_3_gaz', 'consommation_level_4_gaz'], axis=1)

In [83]:
df_combined["counter_statue"] = df_combined["counter_statue"].astype(int)

In [123]:
# Feature engineering (one hot encode all categorical variables)

categorical_variables = ['district','client_catg','region', 'tarif_type','counter_statue','counter_code','reading_remarque'] # all categorical variables
df_combined_encoded = [[]]
df_combined_encoded = pd.get_dummies(df_combined, columns=categorical_variables, drop_first=True) # one hot encode all categorical variables

#df_combined_encoded.info()

In [124]:
# reconvert datetime to numeric

reconvert_to_numeric = ['creation_date','invoice_date']

for col in reconvert_to_numeric:
    df_combined_encoded[col] = pd.to_numeric(df_combined_encoded[col])/100000000000 # convert datetime back into numeric

df_combined_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58069 entries, 0 to 58068
Data columns (total 95 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   client_id                  58069 non-null  object 
 1   creation_date              58069 non-null  float64
 2   invoice_date               58069 non-null  float64
 3   months_number              58069 non-null  int64  
 4   counter_coefficient        58069 non-null  float64
 5   consommation_level_1_elec  58069 non-null  float64
 6   consommation_level_2_elec  58069 non-null  float64
 7   consommation_level_3_elec  58069 non-null  float64
 8   consommation_level_4_elec  58069 non-null  float64
 9   consommation_level_1_gaz   58069 non-null  float64
 10  consommation_level_2_gaz   58069 non-null  float64
 11  district_62                58069 non-null  bool   
 12  district_63                58069 non-null  bool   
 13  district_69                58069 non-null  boo

In [125]:
df_combined_encoded.describe()

,creation_date,invoice_date,months_number,counter_coefficient,consommation_level_1_elec,consommation_level_2_elec,consommation_level_3_elec,consommation_level_4_elec,consommation_level_1_gaz,consommation_level_2_gaz
count,58069.000,58069.000,58069.000,58069.000,58069.000,58069.000,58069.000,58069.000,58069.000,58069.000
mean,10307571.570,12239840.132,100.430,1.001,116.371,42.560,8.251,28.133,23.327,0.000
std,3643045.884,1662743.570,60.613,0.044,155.155,346.872,26.117,337.551,109.118,0.006
min,2239488.000,2376000.000,0.000,0.930,0.000,0.000,0.000,0.000,0.000,0.000
25%,7563456.000,11301120.000,41.000,1.000,61.562,0.000,0.000,0.000,0.000,0.000
50%,11183616.000,11533536.000,112.000,1.000,105.129,1.850,0.000,0.000,0.000,0.000
75%,13306464.000,13421376.000,160.000,1.000,156.727,20.929,1.404,0.000,23.279,0.000
max,15667776.000,15755904.000,179.000,10.000,19602.800,31032.333,200.000,30258.913,6453.957,1.299


In [126]:
df_combined_encoded.columns.value_counts()

client_id               1
counter_code_102.0      1
counter_code_227.0      1
counter_code_222.0      1
counter_code_214.0      1
                       ..
region_308              1
region_307              1
region_306              1
region_305              1
reading_remarque_9.0    1
Name: count, Length: 95, dtype: int64

In [129]:
df_combined_encoded_cols = df_combined_encoded.columns
df_combined_encoded_train_cols = df_combined_encoded_train.columns.drop("target")


In [130]:
# which cols are in df_combined_encoded_cols and not in df_combined_encoded_train_cols
df_combined_encoded_cols[~df_combined_encoded_cols.isin(df_combined_encoded_train_cols)]

Index([], dtype='object')

In [131]:
cols_to_add = list(df_combined_encoded_train_cols[~df_combined_encoded_train_cols.isin(df_combined_encoded_cols)])
cols_to_add

['region_199',
 'tarif_type_30.0',
 'counter_statue_2',
 'counter_statue_618',
 'counter_statue_269375',
 'counter_statue_0',
 'counter_code_5.0',
 'counter_code_367.0',
 'reading_remarque_203.0',
 'reading_remarque_413.0']

In [132]:
# add missing cols and assign the value 0 to them
df_combined_encoded[cols_to_add] = 0

In [133]:
# order cols same order as train dataset

df_combined_encoded = df_combined_encoded[df_combined_encoded_train_cols]

In [142]:
print("Number of cols that do not coincide:", sum(1-(df_combined_encoded.columns == df_combined_encoded_train.columns.drop("target"))))

Number of cols that do not coincide: 0


In [148]:
df_combined_encoded.set_index("client_id",inplace=True)

In [149]:
# Check if any value is infinite
print("Contains Infinity:", np.any(np.isinf(df_combined_encoded)))

# Check if any value is NaN
print("Contains NaN:", np.any(np.isnan(df_combined_encoded)))

Contains Infinity: False
Contains NaN: False


In [162]:
print("Number of cols that do not coincide between X_train and df_combined_encoded:", sum(1-(X_train.columns == df_combined_encoded.columns)))
# drop double columns
X_test = df_combined_encoded.loc[:,~df_combined_encoded.columns.duplicated()]

Number of cols that do not coincide between X_train and df_combined_encoded: 0


In [155]:
# load best boost model
with open('../models/xgboost/xgboost_grid_raw_smart_agg.pkl', 'rb') as f:
    data = pickle.load(f)
best_boost = data["best_boost"]

In [167]:
y_pred_proba = best_boost.predict_proba(X_test)

In [174]:
df_result = pd.DataFrame({"client_id": X_test.index, "target": y_pred_proba[:,1]})
df_result

,client_id,target
0,test_Client_0,0.059
1,test_Client_1,0.224
2,test_Client_10,0.029
3,test_Client_100,0.016
4,test_Client_1000,0.088
...,...,...
58064,test_Client_9995,0.005
58065,test_Client_9996,0.055
58066,test_Client_9997,0.166
58067,test_Client_9998,0.008


In [175]:
df_result.to_csv('../data/test/first_pred.csv', index=False)